In [ ]:
import findspark
findspark.init()

from pyspark import sql, SparkConf, SparkContext

In [ ]:
#conf = SparkConf().setAppName("Read_CSV")
#sc = SparkContext(conf=conf)
#sqlContext = sql.SQLContext(sc)
#
#dfPollution = sqlContext.read.csv("Luchtvervuiling.csv", header=True)

In [ ]:
from prettytable import PrettyTable
import pyparsing as pp
import datetime

#Open de csv file
pollution = open("Luchtvervuiling.csv", 'r')

#Lees de csv file
pollution = pollution.readlines()

#Data opsplitsen op komma, zonder komma's in quotes te verwijderen
def splitDataOnComma(str):
    splitStr = pp.commaSeparatedList.copy().addParseAction(pp.tokenMap(lambda s: s.strip('"')))
    splitStr = splitStr.parseString(str).asList()
    return splitStr

#Headers opsplitsen
headersStr = pollution[0]
headers = splitDataOnComma(headersStr)

#Headers voor tabel
tableHeaders = PrettyTable([headers[12], headers[2], headers[3], headers[5], headers[6], headers[7], headers[8], headers[9]])

# Data toevoegen aan tabel
for i in range(1, 1000) :
    row = splitDataOnComma(pollution[i])
    tableHeaders.add_row([row[12], row[2], row[3], row[5], row[6], row[7], row[8], row[9]])

print(tableHeaders)

In [ ]:
#0 stn_code
#1 sampling_date
#2 state
#3 location
#4 agency
#5 type
#6 so2
#7 no2
#8 rspm
#9 spm
#10 location_monitoring_station
#11 pm2_5
#12 date
#---------------------------------------------------------
#jaar            |12 Date
#maand           |12 Date
#Staat           |2 state
#Stad            |3 location
#Type van gebied |5 type
#SO² (µg/m³)     |6 SO2
#NO² (µg/m³)     |7 NO2
#RSPM (µg/m³)    |8 RSPM
#SPM (µg/m³)     |9 SPM